In [2]:
from pennylane import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt
import qiskit
from qiskit import *
from qiskit import QuantumCircuit, ClassicalRegister
from azure.quantum.qiskit import AzureQuantumProvider
import sklearn.decomposition
from qiskit import transpile
from sklearn import metrics
from mitiq.interface import convert_to_mitiq, convert_from_mitiq

np.set_printoptions(threshold=np.inf)


provider = AzureQuantumProvider(
  resource_id="/subscriptions/430eb3ad-6383-4e03-9396-b1a8ccf6d281/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/Quantu",
  location="westeurope"
)

In [ ]:
num_qubits = 3 


dev = qml.device("default.qubit", wires=num_qubits) 

In [ ]:
np.set_printoptions(threshold=np.inf)


data1 = np.loadtxt("TDS.txt") # import pandas as pd

data2 = np.loadtxt("TLD.txt") #

data3 = np.loadtxt("VDS.txt") #

data4 = np.loadtxt("VLD.txt") #

y_train= data2[:,] 

X_train2=data1[:]

y_test= data4[:,] 

X_test2=data3[:]



pi = np.pi


X_train1 = sklearn.preprocessing.normalize(X_train2, norm='l2',axis=0)

X_test1 = sklearn.preprocessing.normalize(X_test2, norm='l2', axis=0)

X_train = sklearn.preprocessing.normalize(X_train1, norm='l2',axis=1)

X_test = sklearn.preprocessing.normalize(X_test1, norm='l2', axis=1)



In [ ]:
y_train_new =[]


for i in range(len(X_train)):
    if y_train[i]==1:
        a=0.5    # (1-imbalance_ratio)*y_train[i]
        y_train_new.append(a)
    else:
        b=-0.5   # imbalance_ratio*(-1)
        y_train_new.append(b)

In [ ]:
num_qubits = 3
dev = qml.device('default.qubit', wires=num_qubits)

@qml.qnode(dev)
def circuit(params):
    qml.RY(params[0], wires=[0])
    qml.RY(params[1], wires=[1])
    qml.RY(params[2], wires=[2])
    qml.CNOT(wires=[0, 1])
    qml.RY(-params[1], wires=[1])
    qml.CNOT(wires=[0, 2])
    qml.RY(params[3], wires=[2])
    qml.CNOT(wires=[1, 2])
    qml.RY(params[4], wires=[0])
    qml.RY(params[5], wires=[1])
    qml.RY(params[6], wires=[2])
    return qml.state()

def loss(a,prediction):
    total_loss = np.real(np.sum(np.array((a-prediction)**2)))
    return total_loss

def cost(a,params):
    prediction = circuit(params)
    cost = loss(a,prediction)
    return cost


opt = qml.GradientDescentOptimizer(stepsize=0.1)

params = np.array([0.1,0.1,0.1,0.1,0.1,0.1,0.1],requires_grad=True)

y = []
for j in X_train:
    for i in range(200):
        params_and_a,prev_cost = opt.step_and_cost(cost,j,params)
        params = params_and_a[1]
    y.append(params)

In [ ]:
y2 = []
params2 = np.array([0.1,0.1,0.1,0.1,0.1,0.1,0.1],requires_grad=True)
for j in X_test:
    for i in range(200):
        params_and_a,prev_cost = opt.step_and_cost(cost,j,params2)
        params2 = params_and_a[1]
    y2.append(params2)

In [ ]:
def statepreparation(X, circuit):
    
    circuit.ry(X[0], 0)
    circuit.ry(X[1], 1)
    circuit.ry(X[2], 2)
    circuit.cnot(0, 1)
    circuit.ry(-X[1], 1)
    circuit.cnot(0, 2)
    circuit.ry(X[3], 2)
    circuit.cnot(1, 2)
    circuit.ry(X[4], 0)
    circuit.ry(X[5], 1)
    circuit.ry(X[6], 2)
    

    return circuit

In [ ]:
simulator_backend1 = qiskit.Aer.get_backend("ionq.simulator")

In [ ]:
def kernel(feature1, feature2, shots):

    num_qubits1 = 3

    num_qubits2 = 3

    shots = 8192

    circuit1 = QuantumCircuit(num_qubits1)

    circuit2 = QuantumCircuit(num_qubits2)

    circuit1 = statepreparation(feature1, circuit1)
    
    circuit2 = statepreparation(feature2, circuit2,)

    swap_test_circuit = QuantumCircuit(num_qubits1+num_qubits2 + 1, 1)

    swap_test_circuit.compose(circuit1, qubits=[1,2,3], inplace=True )

    swap_test_circuit.compose(circuit2, qubits=[4,5,6], inplace=True )

    swap_test_circuit.h(0)

    swap_test_circuit.cswap(0,1,4)
    swap_test_circuit.cswap(0,2,5)
    swap_test_circuit.cswap(0,3,6)

    swap_test_circuit.h(0)

    swap_test_circuit.measure([0],[0])
    
    decomposed_circuit = qiskit.transpile(swap_test_circuit, basis_gates = ['x', 'h', 'rx','ry' , "cx"])
    # Test conversions
    cirq_circuit, native_type = convert_to_mitiq(decomposed_circuit)
    qiskit_circuit = convert_from_mitiq(cirq_circuit, native_type)

    result = execute(qiskit_circuit,simulator_backend1,shots=shots).result()
    #result = noisy_backend.run(qiskit_circuit, shots=8192).result()
    counts = result.get_counts(qiskit_circuit)
    result=np.zeros(2)
    for key in counts:
        result[int(key,2)]=counts[key]
    result/=shots

    return (2*result[0]-1)

In [ ]:
X1 = [[0.19539425199898733, 0.3321424226379013, 0.48356041495974955, 0.4366236454547067, 1.3429387827168677, 1.7338054045226081, 0.31279502715882584], [0.33894977856091285, 0.1515962332845567, 0.3211028455745527, 0.5680073839460662, 1.2651356455887284, 1.8386401601492575, 0.1407191464955038], [0.45705315136086927, 0.11979880669313199, 0.23416297833146313, 0.3961349974529763, 1.2320421695176043, 1.8528468788739143, 0.05018007614571909], [0.4359381437335388, -0.4292974034918815, 0.20535962703049257, 0.8920730173534486, 0.7579981930833286, 2.131922536523038, -0.012132878204772216], [0.46642582441627295, -0.7030190775601082, 0.24786945590778253, 1.1157280123387523, 0.8216554407322889, 2.329273875198843, -0.08993613729545587], [0.2726685993529912, -0.6266238463171381, 0.29858220085989157, 1.2354001581212255, 0.6505891959586507, 2.2357185070050467, -0.19918817829913738], [0.1721577760419422, -0.4820381419343766, 0.26197782420965854, 1.2923317748091934, 1.4118607264919476, 1.788996391361904, -0.17466794648774553], [0.2537915537036848, -0.4284034681589437, 0.2540162764006526, 1.451330365546301, 1.260883551559626, 1.932244384049667, -0.22119519707505808], [0.2961212561435119, -0.2423521988631969, 0.21181044272258695, 1.254216280442972, 1.3346502145938843, 1.8992076788364358, -0.2727650577702566], [0.09415488681541624, -0.3054554228944181, 0.32520997947614877, 1.407720841436319, 1.517520933872424, 1.6237633647132734, -0.1465511578972676], [0.3878018477741791, -0.4632875426619797, 0.24638793280808768, 1.5463856169004622, 1.2529368224755089, 2.0150667255790458, -0.2803410886476717], [0.143926935132643, -0.44988708538812455, 0.20805993027742634, 1.605845094425583, 1.3399138169523306, 1.7647278627188943, -0.3171492020547146], [0.41961231580705166, -0.20743068208412496, 0.2291473317796239, 0.9884707351488184, 1.3233197204045684, 1.9630464506179128, -0.3052744189497838], [0.3858790347714189, -0.09393263000099035, 0.34259612686344654, 0.7805924666123242, 1.3097989439399076, 1.8551959648101548, -0.19426066462471506], [0.2618635770055782, -0.20355776188748864, 0.392138990477746, 1.1102295922458971, 1.3287156401017093, 1.8146813385508884, -0.1399378565226012], [0.330252593974502, -0.03878966210713623, 0.3082232472193852, 1.0186560701934984, 1.2309948105525863, 1.8626360450252946, -0.23896774417442737], [0.2296937459655206, -0.07401533403829906, 0.284768328019246, 1.112386110910519, 1.0872974353583977, 1.9082780053146562, -0.2654751848581872], [0.21832136779635877, -0.12679120325399618, 0.3014931012956944, 1.2654495629106652, 1.232060127002404, 1.8616221098461612, -0.24647350684680946], [0.22542155591346558, -0.19989743697224707, 0.2528825496809734, 1.3283585400507512, 1.1009176773768594, 1.9242418227351938, -0.3016400536786559], [-0.0472497320115892, -0.19230331658072258, 0.3308376235642885, 1.4473849593621675, 1.659151300659278, 1.5140584358158642, -0.214173876912564], [0.21099206530755463, -0.1076860723106307, 0.29105592584347495, 1.131790467469653, 1.5005606291237927, 1.762377760016982, -0.2554604918203767], [0.19059681940078319, -0.12846680956023981, 0.33436190275508976, 1.1692581230220174, 1.3556521252773899, 1.7784886632481274, -0.21496410393464166], [0.14886524551041866, -0.09873444407211465, 0.33415509623972295, 1.1818698265878331, 1.370303384997075, 1.7628678971086784, -0.21602835667726727], [0.23019305894894865, -0.031171650643481732, 0.34496552193345253, 1.0276202281690998, 1.3876086437739084, 1.7625680148939804, -0.20519174760014747], [0.14966317329275303, 0.01952420824076311, 0.46735739293517886, 0.9466312593044355, 1.5730807438193706, 1.6194436605754794, -0.08214264918681305], [0.15244115076067927, -0.10482672626804627, 0.46381286968413327, 1.1585952747392407, 1.284931165550956, 1.8378222763155079, -0.08350654212891519], [0.3554094584159447, -0.10850214031512971, 0.2883551495450422, 0.9151470336307623, 1.07659530254567, 1.9201398829419265, -0.27045664184881907], [-0.06903201431529735, -0.04221486675811715, 0.5039604403860483, 1.1353488582877902, 1.481160287848428, 1.4694888116638534, -0.05141251681560177], [-0.04600401655496452, -0.009879961766835824, 0.5154324878571661, 1.1545876030689863, 1.578794763357327, 1.5001380953530445, -0.04067840509639966], [0.2017072655814082, -0.01058681138913031, 0.29488525260736353, 1.0127315840679654, 1.0703685250214123, 1.8583529152580243, -0.26181177161323566], [0.4060467871063617, 0.03360392079355041, 0.36803548875524605, 0.8012120459574661, 1.339095543780761, 1.8673947212114026, -0.18890269956173864], [0.3701473277547763, -0.2322601919220693, 0.3231720442058067, 1.020609979340632, 1.0937205057883606, 2.028780426876513, -0.2378511076791631], [0.526600759788444, -0.2189971824037107, 0.3082941005539457, 0.6063401073568897, 1.2401884470031965, 2.081558555430465, -0.2423000990584149], [0.39275213831997957, -0.10356793294185103, 0.35409644986024563, 0.6960941751017878, 1.0943166569085714, 1.832013950636333, -0.2265234001984137], [0.24080907350051564, -0.08964365982938208, 0.3825100268246731, 0.8918787659915635, 1.1121986638182482, 1.875296845326491, -0.19994918617783533], [0.5772728202081113, 0.08171290030988589, 0.39527034962131335, 0.49111970285839834, 1.3888071352106484, 1.9332704919774486, -0.18312101794343905], [0.32186858874160257, 0.0024297872193256836, 0.42014235859799914, 0.7017119455121071, 1.286956270218214, 1.8697545514771883, -0.15673109359311954], [-0.15686388541740462, 0.08229143491507618, 0.7103454074159323, 1.096428240198429, 1.8467596825054877, 1.4223599052266596, 0.13654846037364066], [-0.12483366560864848, 0.17714508070433246, 0.687620032884013, 0.9894287538981175, 1.5830274422080741, 1.5436214596514795, 0.11996505521160206], [-0.0990069084804065, 0.14318724180144768, 0.6181617306007491, 1.1151232809706926, 1.926951090530919, 1.3217048775350404, 0.053508634274711094], [0.2756237989771497, -0.011980799112429746, 0.37516463231404146, 1.0268944636680646, 1.2043317015572819, 1.8984886693746523, -0.18844564318999166], [0.38126813690606004, 0.04922093975977346, 0.35280458482702937, 0.8088703491774003, 1.3159559136363594, 1.8772787192293945, -0.21061900081290094], [0.1640306531002227, 0.08609096827367525, 0.44741271949917855, 0.915635505548861, 1.3710490479503843, 1.765275672063256, -0.11611842017744173], [0.26245512566412393, 0.16688394704804851, 0.388346008400205, 0.7739853576422641, 1.3236905545878106, 1.804853105028199, -0.17257199953414717], [0.06195082837714689, 0.1285027986870249, 0.5498562463624869, 1.0001407399255147, 1.4185838738824137, 1.6870705616327204, -0.01757589890529128], [0.2722937890992426, 0.05866830285779464, 0.3583755121319935, 0.9768715398754532, 1.130844719353412, 1.8916648556910391, -0.20364651069887557], [0.34134528459594876, -0.0582894318561897, 0.42947910917419574, 0.8143213753847873, 1.420447077811752, 1.7335905114341934, -0.13681648559865306], [0.18968946807728054, -0.1498962381130192, 0.46313966341790685, 1.0297461736657887, 1.3533045706861553, 1.787356422887742, -0.10120607561173574], [0.2556334356027741, -0.1695559977237377, 0.35104179646199357, 1.0536321350054092, 1.187851716017528, 1.8995848334160086, -0.22133072287512093], [0.47052836001087367, -0.25514945531892136, 0.3250822293486744, 0.9784231194475774, 1.3006129960290804, 1.9617863848335608, -0.251466466870553], [-0.11283573205732751, -0.15619833885196094, 0.6097316097138894, 1.2820123281748155, 1.77098995818845, 1.4534242088231752, 0.038233178865562795], [0.1383273912399473, -0.16723090197044085, 0.4390008270437674, 1.1840899884179017, 1.4696957413783256, 1.726388866000696, -0.13207377374439436], [0.06542875587178183, -0.13950466547527487, 0.4150751495062038, 1.1074281659716307, 1.5372106453852317, 1.6155092028912228, -0.15632024261835004], [-0.0021373757222833854, -0.1400504138402552, 0.5213429614694733, 1.216440142979816, 1.5818954506326797, 1.6300032308735053, -0.04979534211835392], [0.3670081678058807, -0.055538334113299066, 0.37626646771687794, 0.8487976088331692, 1.3539345032613774, 1.8514071354176174, -0.2013540580766812], [0.3611177851742927, -0.0302377019476916, 0.438396530846566, 0.7214420619770907, 1.4078187886523117, 1.8503843077987498, -0.13733392879546394], [-0.174792497369248, 0.06187596378795292, 0.698745193099557, 1.0740823118266873, 1.7569595115935386, 1.4234935687476475, 0.12568192854787882], [-0.12170228353871408, 0.07603329267035158, 0.5763819500217405, 1.1515021642695005, 1.7732086751811966, 1.4039134817648211, 0.0041906934231876], [0.05038895954101603, 0.09690693171170082, 0.487971073506633, 1.0178443126376262, 1.651740791424626, 1.5113244784728468, -0.08383650076101425], [-0.12296045826387414, 0.10871143384778463, 0.5577692853977715, 1.2201496583770821, 1.7670010560945881, 1.4073343637111428, -0.012641456290008651], [0.2838443435831066, 0.010905181095961888, 0.3851054489381974, 0.8782965857896455, 1.3634723480164992, 1.7286185617668821, -0.18590432468543364], [0.11299809100148765, -0.07692498606525104, 0.4626821195977677, 0.9446377467895423, 1.4147090747211544, 1.5664178222948921, -0.10679983613855985], [0.30160738739951165, -0.05250616062790607, 0.4396832960989014, 0.8066395969218484, 1.4163266398730243, 1.7722357225028573, -0.12839328728305466], [0.3787041202160867, 0.025735482870371272, 0.3826380733768869, 0.6575042174474783, 1.2197471799253983, 1.8855768972562033, -0.18876242558211656], [-0.14296336796102804, 0.14102782372542982, 0.6371972662766281, 0.9834819634619887, 1.6409696354762795, 1.3891219413192006, 0.07116839681970862], [-0.3024663580379835, 0.2959279009987792, 0.4427508263189108, 1.5124867117768797, 1.9438176835374124, 1.1354623168671238, -0.09182679225950661], [0.4070373778523086, 0.05495037358703274, 0.27236651829275565, 0.8767160088266465, 1.3332223880806413, 1.9449021157191573, -0.27903990695610204], [0.22630997422216284, -0.21969429817544264, 0.3523808327876267, 1.292897875198774, 1.1955005955787008, 1.8949584986092478, -0.19331102774375453], [0.459829169831787, -0.08740532915541901, 0.2812538470104569, 0.784723196153229, 1.160496317438704, 1.9612069131482723, -0.2767136965377508], [0.0035733705111611587, -0.08130078048785298, 0.525024314390009, 1.03991683848374, 1.5120270428676208, 1.5159889930820156, -0.026489601455413785], [0.18705963287286853, -0.15292747868775433, 0.37481204119411604, 1.1534707964725406, 0.9865508024443997, 1.9956405591096114, -0.1807143086300782], [-0.13324355914227626, 0.03673875716837527, 0.4574587585942661, 1.1227910170863307, 1.3918478655212136, 1.4989824686607476, -0.1017152723344275], [-0.2029425211338212, 0.21189436457829713, 0.6765089784185337, 1.2028579306320668, 1.677678833679887, 1.5114225790045737, 0.14203936750914412], [-0.06149424216952315, 0.2406770290923135, 0.5747672189982804, 1.142196384989757, 1.764631675101777, 1.4838342066786645, 0.04758673084508309], [0.2882004353074726, 0.046889239598985236, 0.2747711622074744, 1.0220067580352779, 0.9303444047926015, 1.9614598564686871, -0.24001322817742257], [0.3258490192602093, -0.17399962842979103, 0.26972957459484415, 1.2224631301024649, 1.095896166236061, 1.9678474700753859, -0.24837251502040258], [0.18704709988018653, -0.27753113737949514, 0.20679633469395353, 1.3474484607839388, 1.1821709182948084, 1.8329263395091406, -0.3087139445452522], [0.06175955806362893, -0.3183731199113603, 0.2914378840816289, 1.4394668516536724, 1.5960519936910569, 1.552214043559401, -0.2078922906983089], [0.1965889653355582, -0.17572419354717314, 0.20630247351982944, 1.2183402156726058, 1.2359292583595551, 1.8237910613024828, -0.3110205678533876], [0.22104082350950516, -0.22669167343543353, 0.2602100178906863, 1.508675107303049, 1.2264171803566652, 1.9283336816934582, -0.2613601268637849], [0.2980914289952488, -0.0011202410693809232, 0.17844696364496587, 1.0223274535117266, 1.14633100588615, 1.8703913245405086, -0.34892030953666503], [0.35196450647912025, 0.04800440987501034, 0.27055330483129486, 0.907769557358634, 1.133085206259757, 1.8765652912224653, -0.25547151998369655], [0.241464186980949, 0.06157224274766079, 0.3745349385963967, 0.9486208230112444, 1.3735212933976222, 1.794370261407253, -0.15631830994016438], [0.3890363455508678, 0.04791707538338342, 0.3190456159656928, 0.6609891837698993, 1.144757467368949, 1.8695387458569293, -0.20861657527670635], [0.1768342011385598, -0.07852331239963405, 0.45855040052278134, 1.0269592950532207, 1.309494316631167, 1.7919452816875985, -0.07022383558164931], [0.5768441457297981, -0.3268676014245347, 0.2999685962336222, 0.7946232266662241, 1.3347061657459693, 2.099789560331772, -0.229508493717714], [0.27918940702031, -0.09254512113126076, 0.395900926147327, 0.7344701003705097, 1.2829492480769913, 1.789816614471671, -0.15946082682570836], [0.4659753926781317, -0.14330335256869492, 0.36360173646395555, 0.6944295922205608, 1.324702935208732, 1.9693130720482346, -0.18558965107326483], [0.39968157172244, -0.1529923923325561, 0.36484380862132504, 0.7995912895127348, 1.3174531194349288, 1.9632740616416087, -0.1854095296183103], [0.1852958095725119, -0.22884956658084618, 0.45996343963992437, 1.1595613109740839, 1.3623896792696941, 1.8331808417992994, -0.08536336778933068], [0.18991087990197292, -0.1174495282177025, 0.3953013441947644, 0.9456709768615759, 1.4175887075465123, 1.7281972544998359, -0.15291191177023172], [0.13251617072460142, -0.261076528875589, 0.450012518833847, 1.0027588599329083, 1.4326378636922523, 1.5365369992328313, -0.08467497373144162], [0.17550050876601023, -0.23150092618743348, 0.3480115639142299, 0.955892623981498, 1.3146690628204023, 1.7953161188229754, -0.19413711516861457], [0.28189069238692915, -0.08726427549960772, 0.3242068219163955, 1.0171793611033675, 1.2019186594147282, 1.905234851983977, -0.23496118622167095], [0.23366161463294374, -0.12175395056954662, 0.3629949614208188, 1.0752434047890078, 1.294671901333916, 1.79942184291019, -0.19368073217137746], [-0.2755608211366652, 0.16361832354240832, 0.5276072071851308, 1.4284929787737664, 1.8263231433048746, 1.2908794002571031, -0.02395290802480161], [0.11125784292442198, 0.20072345206873154, 0.3319591181452154, 1.1789143819193462, 1.5467321656460185, 1.666006460905875, -0.22251035527741606], [0.2226216628965864, 0.2753025939865961, 0.28419093036751186, 1.0315008001837946, 1.291598592839233, 1.8039205141414356, -0.25874600765493794], [0.1151170429129384, 0.25336725419195155, 0.38058856418895887, 1.1033631718016084, 1.3695646378798587, 1.6812785910243078, -0.16993694638894], [0.1921862485657712, 0.1738006289055539, 0.3636124809073769, 1.1636962427827187, 1.30916427866675, 1.8275750726020628, -0.18992816773924087], [0.0736715805090886, 0.14344686889728345, 0.4031274644761197, 1.1759732553016065, 1.535846782155541, 1.5506867363669248, -0.1583762543314421], [-0.05116513265073221, 0.17130621655341619, 0.47129771782521124, 1.232970511852707, 1.5186047107236567, 1.54588164955196, -0.08791982399694084], [-0.13964003686414425, 0.22612022436969323, 0.5147898768976228, 1.328375837753744, 1.7653219038888888, 1.4132308061860177, -0.03199398692802477], [-0.03955935533918123, 0.2996692192326059, 0.45826047594571184, 1.234433218690097, 1.451142996792923, 1.595986631677207, -0.08723059888745352], [-0.12813327698816104, 0.29125467657407644, 0.6251751660487866, 1.3089679069718878, 1.891441270352339, 1.5097786113433351, 0.09372635410025491], [0.1957439629930569, 0.2289756734507948, 0.37392252560495753, 1.034634570429727, 1.3470635994026379, 1.756263681258922, -0.16220460449006063], [0.09307881668604118, 0.1508898662338106, 0.44047563178220117, 1.0973586349595068, 1.4599477331484456, 1.5572596508830556, -0.10634086387040372], [0.33351128908732697, 0.1323420733446818, 0.35103565642391893, 0.8606494761355057, 1.3040991421502435, 1.8623333225675438, -0.1932898333413734], [-0.15033291957478126, 0.19311238844023484, 0.5985094758210205, 1.1775945179464617, 1.7990041516145348, 1.4285472893823676, 0.053512863191991]]


X3 = [[0.15608126323903493, 0.003098007864123861, 0.33794567653859675, 0.5868645365098715, 0.578530446395731, 2.1968720725943784, 0.3165375470233335], [0.0015572669322274125, 0.020484419627861374, 0.46707571298725187, 0.7339421628255994, 1.3894256582190896, 1.8095432067343402, 0.4451926269659357], [0.24956111048446403, 0.107901193690181, 0.24213161937253974, 0.6329521465398253, 1.3234032866350343, 1.7912731565026712, 0.22150017038273284], [0.0630555298604798, 0.0910472001053859, 0.3969968539516873, 0.76481759957754, 1.5598344040829344, 1.5491206774586521, 0.3711565751641245], [0.11006237951910443, 0.026425713883651735, 0.38223999888484594, 0.7924494704520406, 1.5264865015265108, 1.638066297386283, 0.35510721042420185], [0.14636889684402013, 0.012566403448518598, 0.26358926058523907, 0.8852428309597681, 1.2775621052460309, 1.853998974204334, 0.23715044397652799], [0.09978386959326098, 0.03279756087384991, 0.27515198675490166, 0.8736837143126533, 1.5017088189614156, 1.6236324138352058, 0.24830060665342615], [-0.19854362920201118, 0.08639060847037282, 0.3528045596747519, 1.1836410212764785, 1.8558004033565174, 1.3462461687966083, 0.3287997320970681], [-0.014698428781970906, 0.1184360348538191, 0.2642275044425033, 1.0651848412420795, 1.4898289010132792, 1.6191363049150433, 0.23706392063986148], [0.33760205043471725, 0.15179649008770596, 0.12125441145566455, 0.7273392002497102, 1.292103177995514, 1.763844863473422, 0.09509800396136299], [-0.010838494787395316, 0.14842149030736307, 0.33821319248934284, 0.9336964797555469, 1.5684728853035403, 1.5012022007316106, 0.3043178839907304], [0.34163158451387715, 0.08406226667088174, 0.084901219759306, 0.7097466674167063, 1.1605499843146432, 1.9218092870869374, 0.058583562238646004]]

In [ ]:
def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel(a, b, 8192) for b in B] for a in A])

In [ ]:
def GP(X1, y1, X2):
   
    Σ12 = kernel_matrix(X1, X2) #B
    

    μ =  y1 @ Σ12 

    

    return μ


μ = GP(X1, y_train_new, X3)


C = [] #prediction

for i in μ:

    if i>= 0:
        C.append(1)

    else:
        C.append(0)


print(C)

BB = metrics.confusion_matrix(y_test, C, labels=[0,1])


print(BB)